In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import json

In [2]:
spark = SparkSession.builder.getOrCreate()

In [111]:
config = spark.read.option("multiline","true").json('config.json')
config_df = eval(config.toJSON().collect()[0])

In [39]:
df = spark.read.csv(r'Templates/User - Import Template.csv', header=True)

In [40]:
df.show(truncate=False)

+--------------------------------+----------+----------+
|Email                           |First Name|Last Name |
+--------------------------------+----------+----------+
|jane.doe@bcgasample.com         |Jane      |Doe       |
|alexis.washington@bcgasample.com|Alexis    |Washington|
|reginald.anderson@bcgasample.com|Reginald  |Anderson  |
|tatiana.smithson@bgcasample.com |Tatiana   |Smithson  |
|james.sample@bcgasample.com     |James     |Sample    |
+--------------------------------+----------+----------+



In [167]:
def check_email_format(field):
    regex = r'^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$' 
    df1 = df.filter(df[field].rlike(regex))
    df2 = df.subtract(df1)
    if df2.first():
        return False
    else:
        return True

In [168]:
def validate_null(field):
    if df.filter(df[field].isNull()).first():
        return True
    else:
        return False

In [169]:
def character_check(field_value):
    if df.filter(length(df[field_value[0]])>field_value[1]).first():
        return False
    else:
        return True

In [188]:
res = []
for keys in config_df.keys():
    res.append(list(map(eval(keys),eval(config_df[keys]))))

    result = list(zip(config_df.keys(),res))

In [190]:
result

[('character_check', [True, True]), ('validate_null', [False, False])]

In [156]:
def validate_null(field):
    return df.select(df[field].isNotNull())

In [157]:
def character_check(field, value):
    return df.select(length(df[field])>value)

In [158]:
character_check('First Name',40).distinct().show()

+-------------------------+
|(length(First Name) > 40)|
+-------------------------+
|                    false|
+-------------------------+



In [159]:
validate_null('First Name').distinct().show()

+------------------------+
|(First Name IS NOT NULL)|
+------------------------+
|                    true|
+------------------------+



In [109]:
if validate_null('First Name') == False and validate_null('Last Name') == False and check_email_format('Email') == True:
    print('valid file')
else:
    print('invalid file')

valid file


In [147]:
def validate_null(field):
    
    if df.select(df[field].isNull()).first()[0] == 'False':
        return True
    else:
        return False

In [154]:
def character_check(field_value):
    
    if df.select(length(df[field_value[0]])>field_value[1]).first()[0]:
        return False
    else:
        return True

In [155]:
for keys in config_df.keys():
    print(keys)
    print(config_df[keys])
    print(list(map(eval(keys),eval(config_df[keys]))))

character_check
[('First Name',40),('Last Name',80)]
[True, True]
validate_null
['First Name','Last Name']
[False, False]
